In [2]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import re
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os

# Define the path to your Excel file
excel_file_path = '2018科研人员在线学习数据集.xlsx'

# Load each sheet into a DataFrame
try:
    all_data_df = pd.read_excel(excel_file_path, sheet_name='所有选学数据')
    selected_only_df = pd.read_excel(excel_file_path, sheet_name='仅选学未学')
    selected_and_learned_df = pd.read_excel(excel_file_path, sheet_name='选学且学习')
    print("Data loaded successfully!")
except FileNotFoundError:
    print(f"Error: The Excel file was not found at {excel_file_path}. Please ensure it's in the 'data' directory.")
    # Exit or handle error appropriately in a notebook context
    exit()
except Exception as e:
    print(f"An error occurred during data loading: {e}")
    exit()

# Define output directory for JSON results
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory '{output_dir}' ensured.")



Data loaded successfully!
Output directory 'data' ensured.


In [4]:
import json
from pandas import Timestamp, Interval

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Timestamp):
            return obj.strftime('%Y-%m-%d')
        elif isinstance(obj, Interval):
            return str(obj)
        elif isinstance(obj, (np.integer, int)):
                return int(obj)
        elif isinstance(obj, (np.floating, float)):
            return float(obj)
        elif isinstance(obj, (np.ndarray, list)):
            return obj.tolist()
        return super().default(obj)

In [5]:
# Function to clean and transform a single DataFrame
def clean_and_transform_df(df):
    df.columns = df.columns.str.strip().str.replace('（', '(').str.replace('）', ')').str.replace(' ', '')
    
    if '学习日期' in df.columns:
        df['学习日期'] = pd.to_datetime(df['学习日期'], errors='coerce')
    if '学习时间' in df.columns:
        # Convert to string first to handle mixed types, then to timedelta
        df['学习时间_td'] = df['学习时间'].astype(str).apply(lambda x: pd.to_timedelta(x + ':00') if x.count(':') == 1 else pd.to_timedelta(x, errors='coerce'))
    if '年龄(岁)' in df.columns:
        df['年龄(岁)'] = pd.to_numeric(df['年龄(岁)'], errors='coerce').astype('Int64') 
    if '工作年限(年)' in df.columns:
        df['工作年限(年)'] = pd.to_numeric(df['工作年限(年)'], errors='coerce').astype('Int64')
    if '课程时长(小时)' in df.columns:
        df['课程时长(小时)'] = pd.to_numeric(df['课程时长(小时)'], errors='coerce')
    if '学习时长(小时)' in df.columns:
        df['学习时长(小时)'] = pd.to_numeric(df['学习时长(小时)'], errors='coerce')
    df['职级'] = df['职级'].replace('中级级', '中级')
    # Fill missing numerical values with 0 (adjust strategy if needed)
    for col in ['年龄(岁)', '工作年限(年)', '课程时长(小时)', '学习时长(小时)']:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # Fill missing categorical values with '未知'
    for col in ['性别', '职级', '课程名称']:
        if col in df.columns:
            df[col] = df[col].fillna('未知')
            
    return df

# Apply cleaning to all DataFrames
all_data_df = clean_and_transform_df(all_data_df)
selected_only_df = clean_and_transform_df(selected_only_df)
selected_and_learned_df = clean_and_transform_df(selected_and_learned_df)

print("Data cleaning and transformation complete!")
print("\nSample of '所有选学数据' after cleaning:")
print(all_data_df.head())
print("\nSample of '选学且学习' after cleaning:")
print(selected_and_learned_df.head())

Data cleaning and transformation complete!

Sample of '所有选学数据' after cleaning:
    人员ID 性别  年龄(岁)  工作年限(年)   职级           课程ID              课程名称  课程时长(小时)  \
0  45051  男     54       27  副高级  1493371795041     微积分：8分钟白话（中文）       0.2   
1  45051  男     54       27  副高级  1493374264631  大数据·大智慧——大数据能干什么       0.3   
2  45051  男     54       27  副高级  1514365784843         简约不简单的激光器       0.1   
3  45051  男     54       27  副高级  1514344785197             非线性光学       0.1   
4  45051  男     54       27  副高级  1514363097980           光纤通讯新时代       0.1   

        学习日期      学习时间  学习时长(小时)         学习时间_td  
0 2018-01-01  09:10:16      0.13 0 days 09:10:16  
1 2018-01-01  09:25:14      0.24 0 days 09:25:14  
2 2018-01-01  10:30:25      0.10 0 days 10:30:25  
3 2018-01-01  10:58:15      0.10 0 days 10:58:15  
4 2018-01-01  11:15:51      0.10 0 days 11:15:51  

Sample of '选学且学习' after cleaning:
    人员ID 性别  年龄(岁)  工作年限(年)   职级           课程ID  \
0   3767  男     34        9   中级  1466997088217   
1  

In [6]:

# 总选课人数: Unique Person IDs in '所有选学数据'
total_selected_users = all_data_df['人员ID'].nunique()
total_course = all_data_df['课程ID'].nunique()
# 实际学习人数: Unique Person IDs in '选学且学习'
actual_learning_users = selected_and_learned_df['人员ID'].nunique()

# 选课未学习人数: Unique Person IDs in '仅选学未学'
selected_not_learned_users = actual_learning_users - selected_only_df['人员ID'].nunique()

# 学习转化率 = 实际学习人数 ÷ 总选课人数
learning_conversion_rate = actual_learning_users / total_selected_users if total_selected_users > 0 else 0

# 总选课数量 (records in '所有选学数据')
total_selected_courses_count = len(all_data_df)

# 总学习时长 (sum of '学习时长(小时)' in '选学且学习')
total_learning_duration_hours = selected_and_learned_df['学习时长(小时)'].sum()

# 人均选课数量 (总选课数量 / 独立人员ID数量 in '所有选学数据')
avg_courses_per_total_learner = total_selected_courses_count / total_selected_users if total_selected_users > 0 else 0

# 人均学习时长 (总学习时长 / 独立人员ID数量 in '选学且学习', for actual learners)
avg_learning_duration_per_actual_learner = total_learning_duration_hours / actual_learning_users if actual_learning_users > 0 else 0


overall_participation_results = {
    "total_course": total_course,
    "total_selected_users": total_selected_users,
    "actual_learning_users": actual_learning_users,
    "selected_not_learned_users": selected_not_learned_users,
    "learning_conversion_rate": round(learning_conversion_rate, 4),
    "total_selected_courses_count": total_selected_courses_count,
    "total_learning_duration_hours": round(total_learning_duration_hours, 2),
    "avg_courses_per_total_learner": round(avg_courses_per_total_learner, 2),
    "avg_learning_duration_per_actual_learner": round(avg_learning_duration_per_actual_learner, 2)
}

# Save results to JSON
with open(os.path.join(output_dir, 'overall_participation.json'), 'w', encoding='utf-8') as f:
    json.dump(overall_participation_results, f, indent=4, ensure_ascii=False)

print("Overall participation analysis complete. Results saved to overall_participation.json")
print(json.dumps(overall_participation_results, indent=4, ensure_ascii=False))

Overall participation analysis complete. Results saved to overall_participation.json
{
    "total_course": 1944,
    "total_selected_users": 10360,
    "actual_learning_users": 10245,
    "selected_not_learned_users": 6388,
    "learning_conversion_rate": 0.9889,
    "total_selected_courses_count": 219472,
    "total_learning_duration_hours": 212382.74,
    "avg_courses_per_total_learner": 21.18,
    "avg_learning_duration_per_actual_learner": 20.73
}


In [7]:

demographic_analysis_results = {}

# Prepare 'selected_and_learned_df' for completion calculation
temp_learned_df_for_demog = selected_and_learned_df.copy()
temp_learned_df_for_demog['课程时长(小时)'] = temp_learned_df_for_demog['课程时长(小时)'].replace(0, pd.NA)
temp_learned_df_for_demog['学习完成度'] = temp_learned_df_for_demog['学习时长(小时)'] / temp_learned_df_for_demog['课程时长(小时)']
temp_learned_df_for_demog['学习完成度'] = temp_learned_df_for_demog['学习完成度'].fillna(0).replace([float('inf'), float('-inf')], 0)


def analyze_demographic_group(df_all, df_learned, demographic_col):
    results = {}
    
    if demographic_col not in df_all.columns or demographic_col not in df_learned.columns:
        print(f"Warning: Demographic column '{demographic_col}' not found.")
        return results

    grouped_all = df_all.groupby(demographic_col)['人员ID'].nunique().rename('总选课人数')
    grouped_learned_users = df_learned.groupby(demographic_col)['人员ID'].nunique().rename('实际学习人数')
    
    # Merge and calculate participation rate
    merged_df = pd.merge(grouped_all, grouped_learned_users, left_index=True, right_index=True, how='left').fillna(0)
    merged_df['群体学习参与率'] = merged_df['实际学习人数'] / merged_df['总选课人数']
    merged_df['群体学习参与率'] = merged_df['群体学习参与率'].fillna(0).round(4)

    # Calculate other metrics using the actual learning records
    grouped_learned = df_learned.groupby(demographic_col)
    
    avg_courses_per_person = grouped_learned.apply(lambda x: x.groupby('人员ID')['课程ID'].nunique().mean() if len(x) > 0 else 0).rename('人均学习课程数')
    avg_study_duration_per_person = grouped_learned.apply(lambda x: x['学习时长(小时)'].sum() / x['人员ID'].nunique() if x['人员ID'].nunique() > 0 else 0).rename('人均学习时长')
    avg_completion_rate = grouped_learned['学习完成度'].mean().rename('学习完成度均值')
    course_coverage = grouped_learned.apply(lambda x: x['课程ID'].nunique() / x['人员ID'].nunique() if x['人员ID'].nunique() > 0 else 0).rename('课程覆盖度')

    final_demographic_df = pd.concat([merged_df, avg_courses_per_person, avg_study_duration_per_person, avg_completion_rate, course_coverage], axis=1).fillna(0)
    
    return final_demographic_df.reset_index().to_dict(orient='records')

demographic_analysis_results['gender_analysis'] = analyze_demographic_group(all_data_df, temp_learned_df_for_demog, '性别')
demographic_analysis_results['age_analysis'] = analyze_demographic_group(all_data_df, temp_learned_df_for_demog, '年龄(岁)')
demographic_analysis_results['rank_analysis'] = analyze_demographic_group(all_data_df, temp_learned_df_for_demog, '职级')

# Save results to JSON
with open(os.path.join(output_dir, 'demographic_analysis.json'), 'w', encoding='utf-8') as f:
    json.dump(demographic_analysis_results, f, indent=4, ensure_ascii=False)

print("Demographic analysis complete. Results saved to demographic_analysis.json")
print(json.dumps(demographic_analysis_results, indent=4, ensure_ascii=False))

Demographic analysis complete. Results saved to demographic_analysis.json
{
    "gender_analysis": [
        {
            "性别": "女",
            "总选课人数": 3443,
            "实际学习人数": 3399,
            "群体学习参与率": 0.9872,
            "人均学习课程数": 20.945572227125623,
            "人均学习时长": 20.720785525154458,
            "学习完成度均值": 0.8403034664839184,
            "课程覆盖度": 0.4969108561341571
        },
        {
            "性别": "男",
            "总选课人数": 6917,
            "实际学习人数": 6846,
            "群体学习参与率": 0.9897,
            "人均学习课程数": 19.799298860648555,
            "人均学习时长": 20.735143149284255,
            "学习完成度均值": 0.8363497470845471,
            "课程覆盖度": 0.26628688285130003
        }
    ],
    "age_analysis": [
        {
            "年龄(岁)": 22,
            "总选课人数": 1,
            "实际学习人数": 1,
            "群体学习参与率": 1.0,
            "人均学习课程数": 14.0,
            "人均学习时长": 12.38,
            "学习完成度均值": 0.7859183673469389,
            "课程覆盖度": 14.0
        },
        {
            "年

In [ ]:
print("\n--- Starting Course Preference Analysis ---")

course_preference_results = {}

# 课程学习人数：每门课程在“选学且学习”中的唯一人员ID数
course_learners = selected_and_learned_df.groupby('课程名称')['人员ID'].nunique().sort_values(ascending=False)
course_preference_results['course_learners'] = course_learners.to_dict()

# Temp DataFrame for completion calculations
temp_learned_df_for_course = selected_and_learned_df.copy()
temp_learned_df_for_course['课程时长(小时)'] = temp_learned_df_for_course['课程时长(小时)'].replace(0, pd.NA)
temp_learned_df_for_course['学习完成度'] = temp_learned_df_for_course['学习时长(小时)'] / temp_learned_df_for_course['课程时长(小时)']
temp_learned_df_for_course['学习完成度'] = temp_learned_df_for_course['学习完成度'].fillna(0).replace([float('inf'), float('-inf')], 0)

# 课程完成度均值 = 所有“学习时长 ÷ 课程时长”的平均值
course_completion_avg = temp_learned_df_for_course.groupby('课程名称')['学习完成度'].mean().sort_values(ascending=False)
course_preference_results['course_completion_avg'] = course_completion_avg.round(4).to_dict()

# 高完成课程占比：完成度 > 80% 的课程数占所有课程数量的比例
high_completion_courses = temp_learned_df_for_course[temp_learned_df_for_course['学习完成度'] > 0.8]['课程名称'].nunique()
total_unique_learned_courses = temp_learned_df_for_course['课程名称'].nunique()
course_preference_results['high_completion_course_ratio'] = round(high_completion_courses / total_unique_learned_courses, 4) if total_unique_learned_courses > 0 else 0

# 重复学习率 = 同一用户多次学习同一课程的记录数 ÷ 所有学习记录数
course_user_counts = selected_and_learned_df.groupby(['人员ID', '课程ID']).size()
repeated_studies = course_user_counts[course_user_counts > 1]
num_repeated_records = repeated_studies.apply(lambda x: x - 1).sum()
total_study_records = len(selected_and_learned_df)
course_preference_results['repeated_learning_rate'] = round(num_repeated_records / total_study_records, 4) if total_study_records > 0 else 0

# Word cloud data
def get_word_frequencies(df, column_name):
    text = ' '.join(df[column_name].astype(str).tolist())
    words = re.findall(r'\b\w+\b', text.lower())
    words = [word for word in words if len(word) > 1] # Filter short words
    return Counter(words).most_common(50)
course_preference_results['selected_only_course_word_cloud'] = get_word_frequencies(selected_only_df, '课程名称')
course_preference_results['selected_and_learned_course_word_cloud'] = get_word_frequencies(selected_and_learned_df, '课程名称')

# Top 10热门课程
top_10_popular_courses = course_learners.head(10)
course_preference_results['top_10_popular_courses'] = top_10_popular_courses.to_dict()

# Scatter plot data for Course Duration vs. Actual Learning Duration
scatter_df = temp_learned_df_for_course[['课程时长(小时)', '学习时长(小时)']].dropna()
scatter_df = scatter_df[(scatter_df['课程时长(小时)'] > 0) & (scatter_df['学习时长(小时)'] > 0)]
course_preference_results['course_duration_scatter_data'] = scatter_df.to_dict(orient='records')

# Distribution data for Course Completion Rate
completion_bins = [i/10 for i in range(11)]
completion_distribution = temp_learned_df_for_course['学习完成度'].value_counts(bins=completion_bins, normalize=True).sort_index()
completion_distribution.index = completion_distribution.index.astype(str)
course_preference_results['completion_distribution'] = completion_distribution.round(4).to_dict()


# 选课学习转化率 = 学习过的课程数量 / 所有选学课程数量
unique_learned_courses = selected_and_learned_df['课程ID'].nunique()
unique_selected_courses = all_data_df['课程ID'].nunique() # From all selected data
course_preference_results['course_learning_conversion_rate'] = round(unique_learned_courses / unique_selected_courses, 4) if unique_selected_courses > 0 else 0


# Save results to JSON
with open(os.path.join(output_dir, 'course_preferences.json'), 'w', encoding='utf-8') as f:
    json.dump(course_preference_results, f, indent=4, ensure_ascii=False,cls=CustomJSONEncoder)

print("Course preference and completion analysis complete. Results saved to course_preferences.json")
print(json.dumps(course_preference_results, indent=4, ensure_ascii=False,cls=CustomJSONEncoder))

In [8]:
import jieba
import jieba.analyse
from collections import Counter
import re
course_preference_results_ciyun={}
def process_word_cloud_data(df, column_name):

    # 停用词列表
    stopwords = {
}
    
    # 自定义词典（根据实际课程名称特点添加）
    custom_words = {
        'Python', 'Java', 'C++', 'R语言', '机器学习', '深度学习', '人工智能',
        '数据分析', '数据挖掘', '数据可视化', '统计分析', '回归分析'
    }
    
    # 添加自定义词典
    for word in custom_words:
        jieba.add_word(word)
    
    # 合并所有课程名称
    text = ' '.join(df[column_name].astype(str).tolist())
    
    # 使用jieba进行分词
    words = jieba.cut(text)
    
    # 过滤停用词和无效词
    valid_words = []
    for word in words:
        if (word not in stopwords and  # 不在停用词表中
            len(word.strip()) > 1 and  # 长度大于1
            not word.isdigit() and     # 不是纯数字
            not re.match(r'^[0-9a-zA-Z.]+$', word)):  # 不是纯英文或数字
            valid_words.append(word)
    
    # 统计词频
    word_counts = Counter(valid_words)
    
    # 使用TF-IDF提取关键词
    keywords = jieba.analyse.extract_tags(
        ' '.join(valid_words),
        topK=50,  # 提取前50个关键词
        withWeight=True,  # 返回权重
        allowPOS=('n', 'vn', 'v')  # 只保留名词、动名词和动词
    )
    
    # 生成词云数据
    word_cloud_data = []
    for keyword, weight in keywords:
        if keyword in word_counts:
            word_cloud_data.append({
                "text": keyword,
                "size": int(weight * 1000),  # 将TF-IDF权重转换为合适的显示大小
                "weight": float(f"{weight:.4f}"),  # 保留原始TF-IDF权重
                "frequency": word_counts[keyword]  # 保留词频信息
            })
    
    return word_cloud_data

# 更新course_preference_results中的词云数据
def update_course_preference_results(course_preference_results, selected_only_df, selected_and_learned_df):
    """
    更新课程偏好分析结果
    """
    # 处理仅选学未学的课程词云
    selected_only_cloud = process_word_cloud_data(selected_only_df, '课程名称')
    course_preference_results['selected_only_word_cloud'] = selected_only_cloud
    
    # 处理实际学习的课程词云
    learned_cloud = process_word_cloud_data(selected_and_learned_df, '课程名称')
    course_preference_results['learned_word_cloud'] = learned_cloud
    
    return course_preference_results

# 执行更新
course_preference_results_ciyun = update_course_preference_results(
    course_preference_results_ciyun,
    selected_only_df,
    selected_and_learned_df
)

# 保存更新后的结果
with open(os.path.join(output_dir, 'course_preferences_ciyun.json'), 'w', encoding='utf-8') as f:
    json.dump(course_preference_results_ciyun, f, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)

In [ ]:
print("\n--- Starting Time Behavior Analysis ---")

time_behavior_results = {}

# Ensure datetime columns are correctly parsed for time-based analysis
temp_learned_df_for_time = selected_and_learned_df.copy()
temp_learned_df_for_time.dropna(subset=['学习日期', '学习时间_td'], inplace=True)
temp_learned_df_for_time['学习小时'] = temp_learned_df_for_time['学习时间_td'].dt.components.hours

# 学习活跃时间段分布
bins = [0, 6, 9, 12, 18, 24]
labels = ['深夜(0-6点)', '早晨(6-9点)', '上午(9-12点)', '下午(12-18点)', '晚上(18-24点)']
temp_learned_df_for_time['时间段'] = pd.cut(temp_learned_df_for_time['学习小时'], bins=bins, labels=labels, right=False, include_lowest=True)

time_period_distribution = temp_learned_df_for_time['时间段'].value_counts(normalize=True).sort_index()
time_behavior_results['time_period_distribution'] = time_period_distribution.round(4).to_dict()

# 日活学习人数 
daily_active_users = temp_learned_df_for_time.groupby('学习日期')['人员ID'].nunique()
time_behavior_results['daily_active_users'] = daily_active_users.reset_index().rename(columns={'学习日期': 'date', '人员ID': 'active_users'}).to_dict(orient='records')


# 时间集中度: 统计每人所有学习记录的时间段占比，计算最高占比即为该人员的时间偏好集中度
user_time_preference = {}
for user_id, group in temp_learned_df_for_time.groupby('人员ID'):
    period_counts = group['时间段'].value_counts(normalize=True)
    if not period_counts.empty:
        user_time_preference[user_id] = period_counts.max()
    else:
        user_time_preference[user_id] = 0
time_behavior_results['user_time_concentration_avg'] = round(np.mean(list(user_time_preference.values())), 4)
time_behavior_results['user_time_concentration_raw'] = {str(k): round(v, 4) for k, v in user_time_preference.items()}

# 学习节奏稳定性 = 各学习时间（时间戳）标准差，越低表示越规律。
temp_learned_df_for_time['学习完整时间'] = temp_learned_df_for_time['学习日期'] + temp_learned_df_for_time['学习时间_td']

user_study_rhythm_stability = {}
for user_id, group in temp_learned_df_for_time.groupby('人员ID'):
    if len(group) > 1:
        timestamps = group['学习完整时间'].apply(lambda x: x.timestamp()).sort_values()
        user_study_rhythm_stability[user_id] = timestamps.std()
    else:
        user_study_rhythm_stability[user_id] = 0
time_behavior_results['user_study_rhythm_stability_avg'] = round(np.mean(list(user_study_rhythm_stability.values())), 2)
time_behavior_results['user_study_rhythm_stability_raw'] = {str(k): round(v, 2) for k, v in user_study_rhythm_stability.items()}

# Calendar heatmap data (weekday vs hour)
temp_learned_df_for_time['weekday'] = temp_learned_df_for_time['学习日期'].dt.dayofweek
temp_learned_df_for_time['hour'] = temp_learned_df_for_time['学习小时']

calendar_heatmap_data = temp_learned_df_for_time.groupby(['weekday', 'hour']).size().unstack(fill_value=0)
heatmap_list = []
for weekday in calendar_heatmap_data.index:
    for hour in calendar_heatmap_data.columns:
        heatmap_list.append({
            "weekday": int(weekday),
            "hour": int(hour),
            "count": int(calendar_heatmap_data.loc[weekday, hour])
        })
time_behavior_results['calendar_heatmap_data'] = heatmap_list

# Save results to JSON
with open(os.path.join(output_dir, 'time_behavior.json'), 'w', encoding='utf-8') as f:
    json.dump(time_behavior_results, f, indent=4, ensure_ascii=False,cls=CustomJSONEncoder)

print("Learning time behavior analysis complete. Results saved to time_behavior.json")
print(json.dumps(time_behavior_results, indent=4, ensure_ascii=False,cls=CustomJSONEncoder))

In [ ]:
print("\n--- Starting Quality and Efficiency Analysis ---")

quality_efficiency_results = {}

temp_learned_df_for_quality = selected_and_learned_df.copy()
temp_learned_df_for_quality['课程时长(小时)'] = temp_learned_df_for_quality['课程时长(小时)'].replace(0, pd.NA)
temp_learned_df_for_quality['学习完成度'] = temp_learned_df_for_quality['学习时长(小时)'] / temp_learned_df_for_quality['课程时长(小时)']
temp_learned_df_for_quality['学习完成度'] = temp_learned_df_for_quality['学习完成度'].fillna(0).replace([float('inf'), float('-inf')], 0)

# Ensure '学习完整时间' for interval calculation
temp_learned_df_for_quality['学习完整时间'] = temp_learned_df_for_quality['学习日期'] + temp_learned_df_for_quality['学习时间_td']
temp_learned_df_for_quality.dropna(subset=['学习完成度', '学习完整时间'], inplace=True)

# 平均完成度 = 学习时长 ÷ 课程时长
quality_efficiency_results['average_completion_rate'] = round(temp_learned_df_for_quality['学习完成度'].mean(), 4)

# 高完成记录占比：完成度 >80% 的记录数 ÷ 总学习记录数
high_completion_records = len(temp_learned_df_for_quality[temp_learned_df_for_quality['学习完成度'] > 0.8])
total_records = len(temp_learned_df_for_quality)
quality_efficiency_results['high_completion_record_ratio'] = round(high_completion_records / total_records, 4) if total_records > 0 else 0

# 放弃率：完成度 <20% 的记录占比
abandonment_records = len(temp_learned_df_for_quality[temp_learned_df_for_quality['学习完成度'] < 0.2])
quality_efficiency_results['abandonment_rate'] = round(abandonment_records / total_records, 4) if total_records > 0 else 0

# 平均每次学习时长：所有学习时长之和 ÷ 记录总数
quality_efficiency_results['average_duration_per_study_session'] = round(temp_learned_df_for_quality['学习时长(小时)'].mean(), 4)

# 平均学习间隔：按每人学习时间排序，计算相邻两次的平均时间间隔。
user_avg_intervals = {}
for user_id, group in temp_learned_df_for_quality.groupby('人员ID'):
    if len(group) > 1:
        sorted_times = group['学习完整时间'].sort_values()
        intervals = sorted_times.diff().dropna().dt.total_seconds() / (60 * 60)
        if not intervals.empty:
            user_avg_intervals[user_id] = intervals.mean()
        else:
            user_avg_intervals[user_id] = 0
    else:
        user_avg_intervals[user_id] = 0

quality_efficiency_results['average_learning_interval_hours'] = round(np.mean(list(user_avg_intervals.values())), 2)
quality_efficiency_results['user_learning_intervals_raw'] = {str(k): round(v, 2) for k,v in user_avg_intervals.items()}

# Histograms for learning completion distribution
completion_bins = [i/10 for i in range(11)]
completion_counts, _ = np.histogram(temp_learned_df_for_quality['学习完成度'].dropna(), bins=completion_bins)
quality_efficiency_results['completion_histogram'] = {f"{completion_bins[i]}-{completion_bins[i+1]}": int(completion_counts[i]) for i in range(len(completion_counts))}

# Box plot data for high completion vs. low completion groups
high_completion_group = temp_learned_df_for_quality[temp_learned_df_for_quality['学习完成度'] > 0.8]
low_completion_group = temp_learned_df_for_quality[temp_learned_df_for_quality['学习完成度'] < 0.2]

box_plot_data = {
    'high_completion_group': {
        'age': high_completion_group['年龄(岁)'].dropna().tolist(),
        'work_experience': high_completion_group['工作年限(年)'].dropna().tolist(),
        'study_duration': high_completion_group['学习时长(小时)'].dropna().tolist()
    },
    'low_completion_group': {
        'age': low_completion_group['年龄(岁)'].dropna().tolist(),
        'work_experience': low_completion_group['工作年限(年)'].dropna().tolist(),
        'study_duration': low_completion_group['学习时长(小时)'].dropna().tolist()
    }
}
quality_efficiency_results['box_plot_data'] = box_plot_data

# Scatter plot for completion vs. course duration
scatter_df = temp_learned_df_for_quality[['学习完成度', '课程时长(小时)']].dropna()
scatter_df = scatter_df[scatter_df['课程时长(小时)'] > 0]
quality_efficiency_results['completion_vs_course_duration_scatter'] = scatter_df.to_dict(orient='records')

# Save results to JSON
with open(os.path.join(output_dir, 'quality_efficiency.json'), 'w', encoding='utf-8') as f:
    json.dump(quality_efficiency_results, f, indent=4, ensure_ascii=False,cls=CustomJSONEncoder)

print("Learning quality and efficiency analysis complete. Results saved to quality_efficiency.json")
print(json.dumps(quality_efficiency_results, indent=4, ensure_ascii=False,cls=CustomJSONEncoder))

In [ ]:
print("\n--- Starting Clustering Analysis ---")

clustering_results = {}

temp_learned_df_for_clustering = selected_and_learned_df.copy()
temp_learned_df_for_clustering['课程时长(小时)'] = temp_learned_df_for_clustering['课程时长(小时)'].replace(0, pd.NA)
temp_learned_df_for_clustering['学习完成度'] = temp_learned_df_for_clustering['学习时长(小时)'] / temp_learned_df_for_clustering['课程时长(小时)']
temp_learned_df_for_clustering['学习完成度'] = temp_learned_df_for_clustering['学习完成度'].fillna(0).replace([float('inf'), float('-inf')], 0)

temp_learned_df_for_clustering['学习完整时间'] = temp_learned_df_for_clustering['学习日期'] + temp_learned_df_for_clustering['学习时间_td']
temp_learned_df_for_clustering.dropna(subset=['学习完成度', '学习完整时间'], inplace=True)

# 1. Feature Engineering
features_df = pd.DataFrame(index=temp_learned_df_for_clustering['人员ID'].unique())

# 学习频率维度: 学习记录数
features_df['学习记录数'] = temp_learned_df_for_clustering.groupby('人员ID').size()

# 学习投入维度: 总学习时长（小时）, 平均单次学习时长
features_df['总学习时长(小时)'] = temp_learned_df_for_clustering.groupby('人员ID')['学习时长(小时)'].sum()
features_df['平均单次学习时长'] = temp_learned_df_for_clustering.groupby('人员ID')['学习时长(小时)'].mean()

# 学习完成度维度: 平均完成度, 高完成率, 放弃率
features_df['平均完成度'] = temp_learned_df_for_clustering.groupby('人员ID')['学习完成度'].mean()
high_completion_records_by_user = temp_learned_df_for_clustering[temp_learned_df_for_clustering['学习完成度'] >= 0.8].groupby('人员ID').size()
features_df['高完成率'] = high_completion_records_by_user / features_df['学习记录数']
features_df['高完成率'] = features_df['高完成率'].fillna(0) # If no high completion records for a user
abandonment_records_by_user = temp_learned_df_for_clustering[temp_learned_df_for_clustering['学习完成度'] < 0.2].groupby('人员ID').size()
features_df['放弃率'] = abandonment_records_by_user / features_df['学习记录数']
features_df['放弃率'] = features_df['放弃率'].fillna(0) # If no abandonment records for a user

# 学习节奏与习惯维度: 学习时间标准差, 晚间学习占比
user_time_std = {}
for user_id, group in temp_learned_df_for_clustering.groupby('人员ID'):
    if len(group) > 1:
        timestamps = group['学习完整时间'].apply(lambda x: x.timestamp()).sort_values()
        user_time_std[user_id] = timestamps.std()
    else:
        user_time_std[user_id] = 0
features_df['学习时间标准差'] = features_df.index.map(user_time_std).fillna(0)

temp_learned_df_for_clustering['学习小时'] = temp_learned_df_for_clustering['学习时间_td'].dt.components.hours
evening_study_by_user = temp_learned_df_for_clustering[(temp_learned_df_for_clustering['学习小时'] >= 18) & (temp_learned_df_for_clustering['学习小时'] < 24)].groupby('人员ID').size()
features_df['晚间学习占比'] = evening_study_by_user / features_df['学习记录数']
features_df['晚间学习占比'] = features_df['晚间学习占比'].fillna(0)

# 多样性维度: 课程种类数
features_df['课程种类数'] = temp_learned_df_for_clustering.groupby('人员ID')['课程ID'].nunique()

# 重复学习维度: 重复学习率
user_repeat_rate = {}
for user_id, group in temp_learned_df_for_clustering.groupby('人员ID'):
    if len(group) > 0:
        course_counts = group['课程ID'].value_counts()
        num_repeated_records_user = course_counts[course_counts > 1].apply(lambda x: x - 1).sum()
        user_repeat_rate[user_id] = num_repeated_records_user / len(group)
    else:
        user_repeat_rate[user_id] = 0
features_df['重复学习率'] = features_df.index.map(user_repeat_rate).fillna(0)

# Fill any remaining NaNs (e.g., from users with single records where std dev is undefined)
features_df = features_df.fillna(0)

print("Feature engineering complete.")
print("\nSample of engineered features:")
print(features_df.head())


# 2. Clustering Method (KMeans)
n_clusters = 5 # Example: 5 clusters, can be optimized later

if not features_df.empty:
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features_df)
    scaled_features_df = pd.DataFrame(scaled_features, columns=features_df.columns, index=features_df.index)

    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    features_df['cluster'] = kmeans.fit_predict(scaled_features)

    # PCA for visualization (reduce to 2 components)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_features)
    pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'], index=features_df.index)
    pca_df['cluster'] = features_df['cluster']
    feature_columns = features_df.columns[:-1] if 'cluster' in features_df.columns else features_df.columns
    # Analyze cluster characteristics (centroids)
    cluster_centroids_original_scale = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=feature_columns)
    cluster_centroids_original_scale['cluster'] = range(n_clusters)
    
    clustering_results = {
        "cluster_assignments": features_df['cluster'].reset_index().rename(columns={'人员ID': 'person_id', 'cluster': 'cluster'}).to_dict(orient='records'),
        "pca_data": pca_df.reset_index().rename(columns={'人员ID': 'person_id'}).to_dict(orient='records'),
        "cluster_centroids_original_scale": cluster_centroids_original_scale.round(4).to_dict(orient='records')
    }
else:
    print("No features generated for clustering.")


# Save results to JSON
with open(os.path.join(output_dir, 'clustering_results.json'), 'w', encoding='utf-8') as f:
    json.dump(clustering_results, f, indent=4, ensure_ascii=False)

print("Clustering analysis complete. Results saved to clustering_results.json")
print(json.dumps(clustering_results, indent=4, ensure_ascii=False))